In [1]:
# Imports
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
import time
import sys
import chromedriver_binary
import re
from collections import defaultdict
from glob import glob
import pickle

In [2]:
def fmt_url(url_string, items_per_page, page_nbr):
    page = (items_per_page * (page_nbr - 1))
    print('Getting page:', page)
    return url_string.format(page)

In [3]:
def get_property_info(driver, link):
    driver.switch_to.window(driver.window_handles[0])
    driver.get(link)
    time.sleep(10)
    soup = BeautifulSoup(driver.page_source, "lxml")
    return soup

# Example
# soup = get_property_info(driver, 'https://www.airbnb.com/')

In [4]:
def get_attrs_1_4(div_list):
    attrs_dict = {}
    res_dict = {'guests' : re.compile(r'^\d+ guests'),
                'br' : re.compile(r'^\d+ bedrooms?'),
                'beds' : re.compile(r'^\d+ beds?'),
                'baths' : re.compile(r'^\d+ baths?')}
    for div in div_list:
        for key, val in res_dict.items():
            if val.match(div.text):
                attrs_dict[key] = div.text
    return attrs_dict

# Example
# get_attrs_1_4(soup.find_all('div', {'class' : '_czm8crp'}))

In [5]:
def get_price(span):
    print(span)
    return span.text

# Example
# get_price(soup.find('span', {'class' : '_doc79r'}))

In [6]:
def get_attrs_5_6(spans):
    attrs_dict = {}
    spans_len = len(spans)
    for idx, span in enumerate(spans):
        if span.text == 'Cleaning fee' and idx + 1 <= spans_len:
            attrs_dict['clean_fee'] = spans[idx + 1].text
        elif span.text == 'Service fee' and idx + 1 <= spans_len:
            attrs_dict['serv_fee'] = spans[idx + 1].text
        else:
            pass
    return attrs_dict

# Eample
# get_attrs_5_6(soup.find_all('span', {'class' : '_1jlnvra2'}))

In [7]:
def get_amenities(buttons):
    amentities = ''
    ams = re.compile(r'^Show all \d+ amenities')
    for button in buttons:
        if ams.match(button.text):
            amenities = button.text
    return amenities

# Example
# get_amenities(soup.find_all('button', {'class' : '_b0ybw8s'}))

In [8]:
def get_lat_lon(scripts):
    lat_lon = ''
    ll = re.compile(r'ViewportInfoService.GetViewportInfo')
    parse_ll = re.compile(r';1d47\.\d{14}&amp;2d-122\.\d{14}')
    for script in scripts:
        if ll.search(script.attrs['src']):
            lat_lon = script.attrs['src']
            # print(script.attrs['src'])
    return lat_lon

# Example
# get_lat_lon(soup.find_all('script',  {'charset' : "UTF-8"}))

In [9]:
def get_superhost(spans):
    superhost = False
    sh = re.compile(r'^\w+ is a Superhost')
    for span in spans:
        if sh.match(span.text):
            superhost = True
    return superhost

# Example
# get_superhost(soup.find_all('span', {'class' : "_1p3joamp"}))

In [10]:
def get_links(driver, url):
    list_of_links = []
    for i in range(1, 6):
        url = fmt_url(url, 18, i)
        # print(url)
        driver.switch_to.window(window)
        driver.get(url)
        time.sleep(10)
        dict_of_urls = defaultdict(int)
        a = driver.find_elements_by_tag_name('a')
    for element in a:
        link = element.get_attribute('href')
        if not link is None and link.find('rooms') != -1:
            dict_of_urls[link] += 1
    list_of_links += list(dict_of_urls.keys())
    return list(dict_of_urls.keys())

# Example
# get_links(driver, URL, 1)

In [11]:
# URLS we will pass in to get links to individual properties
URL_1_GUEST = 'https://www.airbnb.com/s/Seattle--WA--United-States/homes?refinement_paths%5B%5D=%2Fhomes&current_tab_id=home_tab&selected_tab_id=home_tab&place_id=ChIJVTPokywQkFQRmtVEaUZlJRA&search_type=pagination&screen_size=large&hide_dates_and_guests_filters=false&checkin=2019-12-22&checkout=2019-12-28&adults=1&room_types%5B%5D=Entire%20home%2Fapt&s_tag=hw8cp5Pu&section_offset=5&items_offset={}&last_search_session_id=788955f0-95fc-4576-8543-ed1dcfe94ed3'
URL_2_GUEST = 'https://www.airbnb.com/s/Seattle--WA--United-States/homes?refinement_paths%5B%5D=%2Fhomes&current_tab_id=home_tab&selected_tab_id=home_tab&place_id=ChIJVTPokywQkFQRmtVEaUZlJRA&search_type=pagination&screen_size=large&hide_dates_and_guests_filters=false&checkin=2019-12-22&checkout=2019-12-28&adults=2&room_types%5B%5D=Entire%20home%2Fapt&s_tag=M4qG3z8c&section_offset=5&items_offset={}&last_search_session_id=40094e9c-6a7b-4bf5-a60d-ad757e06c4bf'
URL_3_GUEST = 'https://www.airbnb.com/s/Seattle--WA--United-States/homes?refinement_paths%5B%5D=%2Fhomes&current_tab_id=home_tab&selected_tab_id=home_tab&place_id=ChIJVTPokywQkFQRmtVEaUZlJRA&search_type=pagination&screen_size=large&hide_dates_and_guests_filters=false&checkin=2019-12-22&checkout=2019-12-28&adults=3&room_types%5B%5D=Entire%20home%2Fapt&s_tag=TNcnvTMZ&section_offset=5&items_offset={}&last_search_session_id=161c0850-d38d-459a-a90d-91cab7b2dff7'
URL_4_GUEST = 'https://www.airbnb.com/s/Seattle--WA--United-States/homes?refinement_paths%5B%5D=%2Fhomes&current_tab_id=home_tab&selected_tab_id=home_tab&place_id=ChIJVTPokywQkFQRmtVEaUZlJRA&search_type=pagination&screen_size=large&hide_dates_and_guests_filters=false&checkin=2019-12-22&checkout=2019-12-28&adults=4&room_types%5B%5D=Entire%20home%2Fapt&s_tag=NCdHnnOK&section_offset=5&items_offset={}&last_search_session_id=45e6e542-db96-4fc2-8eba-7d58709bcb1e'
url_list = [URL_1_GUEST, URL_2_GUEST, URL_3_GUEST, URL_4_GUEST]

In [12]:
# Create Selenium chrome browser driver instance, and an empty list to hold links
list_of_links = []
driver = webdriver.Chrome()
time.sleep(5)
window = driver.window_handles[0]

In [13]:
for url in url_list:
    list_of_links += get_links(driver, url)

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=77.0.3865.90)


In [ ]:
!pwd

In [ ]:
with open('../data/long_list_of_links.txt', 'w+')

In [ ]:
pickle.dump(list_of_links, '../data/long_list_of_links.txt')

In [ ]:
for i in range(1, 18):
    url2 = fmt_url(URL, 18, i)
    driver.switch_to.window(window)
    driver.get(url2)
    time.sleep(10)
    dict_of_urls = defaultdict(int)
    a = driver.find_elements_by_tag_name('a')
    for element in a:
        link = element.get_attribute('href')
        if not link is None and link.find('rooms') != -1:
            dict_of_urls[link] += 1
    list_of_links += list(dict_of_urls.keys())

In [ ]:
print(len(list_of_links))
list_of_links[:5]

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(pd.Series(list_of_links), columns=['links']).to_csv('links3.csv', index=False)

In [ ]:
list_of_links[0] 

In [ ]:
link_files = glob('/Users/scott/metis/projects/project2/notebooks/links*')
link_files

In [ ]:
links_df = pd.concat([pd.read_csv(file) for file in link_files])

In [ ]:
links_df.head()

In [ ]:
links_df.drop_duplicates(inplace=True)

In [ ]:
links_df.shape

In [ ]:
type(links_df.iloc[0][0])

In [ ]:
xpaths = {'guests' : '//*[@id="room"]/div[2]/div/div[2]/div[1]/div/div[3]/div/div/div/div[1]/div[2]/div[2]/div[1]/div',
          'bedrooms' : '//*[@id="room"]/div[2]/div/div[2]/div[1]/div/div[3]/div/div/div/div[1]/div[2]/div[2]/div[2]/div',
          'beds' : '//*[@id="room"]/div[2]/div/div[2]/div[1]/div/div[3]/div/div/div/div[1]/div[2]/div[2]/div[3]/div',
          'baths' : '//*[@id="room"]/div[2]/div/div[2]/div[1]/div/div[3]/div/div/div/div[1]/div[2]/div[2]/div[4]/div',
          'narrative' : '//*[@id="details"]/div/div[1]/div/div/div/p/span/span',
          'amenities' : '//*[@id="amenities"]/div/div/div/div/div/section/div[4]/div/button',
          'price' : '//*[@id="room"]/div[2]/div/div[2]/div[2]/div/div/div[1]/div/div/div[1]/div/div/div[2]/div[1]/div/span[2]/span',
          'rules' : '//*[@id="house-rules"]/div/section/div[3]/div[1]/div/div/div[2]/div[2]',
          'cleaning_fee' : '//*[@id="book_it_form"]/div[2]/div[2]/div[1]/div[2]/span/span',
          'map' : '//*[@id="neighborhood"]/div/div/div/div/div/section/div[2]/div[2]/div[2]/img'
          }

In [ ]:
driver = webdriver.Chrome()

In [ ]:
[re.compile(r'^\d+ guests'), re.compile(r'^\d+ bedrooms?'), re.compile(r'^\d+ beds?'), re.compile(r'^\d+ baths?')]

In [ ]:
guests = re.compile(r'^\d+ guests')
guests.match('2 guests')

In [ ]:
br = re.compile(r'^\d+ bedrooms?')
br.match('6 bedrooms') and 'hello'

In [ ]:
beds = re.compile(r'^\d+ beds?')
beds.match('2 beds') and 'Yes!'

In [ ]:
baths = re.compile(r'^\d+ baths?')
baths.match('6 baths') and 'Yes!'

In [ ]:
# Price :: Y value
# <span class="_doc79r">$109</span>
soup.find('span', {'class' : '_doc79r'})

In [ ]:
# 5 Cleaning fee
# 6 Service Fee
# <div style="margin-bottom: 8px;"><div><div class="_hgs47m"><div class="_10ejfg4u"><span class="_1jlnvra2"><span>$109 x 6 nights</span></span><div class="_1c2cbn7k"><div class="_jx9fdbv" role="presentation"><div role="button" tabindex="-1" aria-expanded="false"><button aria-expanded="false" aria-label="Learn more" type="button" class="_pmxdr5"><span class="_3hmsj"><svg viewBox="0 0 24 24" role="presentation" aria-hidden="true" focusable="false" style="height: 15px; width: 15px; display: block; fill: rgb(72, 72, 72);"><path d="m12 0c-6.63 0-12 5.37-12 12s5.37 12 12 12 12-5.37 12-12-5.37-12-12-12zm0 23c-6.07 0-11-4.92-11-11s4.93-11 11-11 11 4.93 11 11-4.93 11-11 11zm4.75-14c0 1.8-.82 2.93-2.35 3.89-.23.14-1 .59-1.14.67-.4.25-.51.38-.51.44v2a .75.75 0 0 1 -1.5 0v-2c0-.74.42-1.22 1.22-1.72.17-.11.94-.55 1.14-.67 1.13-.71 1.64-1.41 1.64-2.61a3.25 3.25 0 0 0 -6.5 0 .75.75 0 0 1 -1.5 0 4.75 4.75 0 0 1 9.5 0zm-3.75 10a1 1 0 1 1 -2 0 1 1 0 0 1 2 0z" fill-rule="evenodd"></path></svg></span></button></div></div></div></div><div class="_ni9axhe"><span class="_1jlnvra2"><span class="_j1kt73">$655</span></span></div></div><div style="margin-top: 8px; margin-bottom: 8px;"><div class="_7qp4lh"></div></div></div><div><div class="_hgs47m"><div class="_10ejfg4u"><span class="_1jlnvra2"><span>Cleaning fee</span></span><div class="_1c2cbn7k"><div class="_jx9fdbv" role="presentation"><div role="button" tabindex="-1" aria-expanded="false"><button aria-expanded="false" aria-label="Learn more" type="button" class="_pmxdr5"><span class="_3hmsj"><svg viewBox="0 0 24 24" role="presentation" aria-hidden="true" focusable="false" style="height: 15px; width: 15px; display: block; fill: rgb(72, 72, 72);"><path d="m12 0c-6.63 0-12 5.37-12 12s5.37 12 12 12 12-5.37 12-12-5.37-12-12-12zm0 23c-6.07 0-11-4.92-11-11s4.93-11 11-11 11 4.93 11 11-4.93 11-11 11zm4.75-14c0 1.8-.82 2.93-2.35 3.89-.23.14-1 .59-1.14.67-.4.25-.51.38-.51.44v2a .75.75 0 0 1 -1.5 0v-2c0-.74.42-1.22 1.22-1.72.17-.11.94-.55 1.14-.67 1.13-.71 1.64-1.41 1.64-2.61a3.25 3.25 0 0 0 -6.5 0 .75.75 0 0 1 -1.5 0 4.75 4.75 0 0 1 9.5 0zm-3.75 10a1 1 0 1 1 -2 0 1 1 0 0 1 2 0z" fill-rule="evenodd"></path></svg></span></button></div></div></div></div><div class="_ni9axhe"><span class="_1jlnvra2"><span class="_j1kt73">$75</span></span></div></div><div style="margin-top: 8px; margin-bottom: 8px;"><div class="_7qp4lh"></div></div></div><div><div class="_hgs47m"><div class="_10ejfg4u"><span class="_1jlnvra2"><span>Service fee</span></span><div class="_1c2cbn7k"><div class="_jx9fdbv" role="presentation"><div role="button" tabindex="-1" aria-expanded="false"><button aria-expanded="false" aria-label="Learn more" type="button" class="_pmxdr5"><span class="_3hmsj"><svg viewBox="0 0 24 24" role="presentation" aria-hidden="true" focusable="false" style="height: 15px; width: 15px; display: block; fill: rgb(72, 72, 72);"><path d="m12 0c-6.63 0-12 5.37-12 12s5.37 12 12 12 12-5.37 12-12-5.37-12-12-12zm0 23c-6.07 0-11-4.92-11-11s4.93-11 11-11 11 4.93 11 11-4.93 11-11 11zm4.75-14c0 1.8-.82 2.93-2.35 3.89-.23.14-1 .59-1.14.67-.4.25-.51.38-.51.44v2a .75.75 0 0 1 -1.5 0v-2c0-.74.42-1.22 1.22-1.72.17-.11.94-.55 1.14-.67 1.13-.71 1.64-1.41 1.64-2.61a3.25 3.25 0 0 0 -6.5 0 .75.75 0 0 1 -1.5 0 4.75 4.75 0 0 1 9.5 0zm-3.75 10a1 1 0 1 1 -2 0 1 1 0 0 1 2 0z" fill-rule="evenodd"></path></svg></span></button></div></div></div></div><div class="_ni9axhe"><span class="_1jlnvra2"><span class="_j1kt73">$94</span></span></div></div><div style="margin-top: 8px; margin-bottom: 8px;"><div class="_7qp4lh"></div></div></div><div><div class="_hgs47m"><div class="_10ejfg4u"><span class="_1jlnvra2"><span>Occupancy taxes and fees</span></span><div class="_1c2cbn7k"><div class="_jx9fdbv" role="presentation"><div role="button" tabindex="-1" aria-expanded="false"><button aria-expanded="false" aria-label="Learn more" type="button" class="_pmxdr5"><span class="_3hmsj"><svg viewBox="0 0 24 24" role="presentation" aria-hidden="true" focusable="false" style="height: 15px; width: 15px; display: block; fill: rgb(72, 72, 72);"><path d="m12 0c-6.63 0-12 5.37-12 12s5.37 12 12 12 12-5.37 12-12-5.37-12-12-12zm0 23c-6.07 0-11-4.92-11-11s4.93-11 11-11 11 4.93 11 11-4.93 11-11 11zm4.75-14c0 1.8-.82 2.93-2.35 3.89-.23.14-1 .59-1.14.67-.4.25-.51.38-.51.44v2a .75.75 0 0 1 -1.5 0v-2c0-.74.42-1.22 1.22-1.72.17-.11.94-.55 1.14-.67 1.13-.71 1.64-1.41 1.64-2.61a3.25 3.25 0 0 0 -6.5 0 .75.75 0 0 1 -1.5 0 4.75 4.75 0 0 1 9.5 0zm-3.75 10a1 1 0 1 1 -2 0 1 1 0 0 1 2 0z" fill-rule="evenodd"></path></svg></span></button></div></div></div></div><div class="_ni9axhe"><span class="_1jlnvra2"><span class="_j1kt73">$129</span></span></div></div><div style="margin-top: 8px; margin-bottom: 8px;"><div class="_7qp4lh"></div></div></div><div><div class="_hgs47m"><div class="_10ejfg4u"><span class="_121z06r2"><span>Total</span></span></div><div class="_ni9axhe"><span class="_121z06r2"><span class="_j1kt73">$953</span></span></div></div></div></div>
soup.find_all('span', {'class' : '_1jlnvra2'})

In [ ]:
# 7 Amenities
# <button type="button" class="_b0ybw8s" aria-busy="false">Show all 32 amenities</button>
soup.find_all('button', {'class' : '_b0ybw8s'})

In [ ]:
re.compile(r'^Show all \d+ amenities').match('Show all 32 amenities')

In [ ]:
soup.find_all('script',  {'charset' : "UTF-8"})[8].attrs['src']

In [ ]:
# superhost
# <span class="_1p3joamp">Emma is a Superhost</span>
soup.find_all('span', {'class' : "_1p3joamp"})

In [ ]:
ll = re.compile(r'ViewportInfoService.GetViewportInfo')
ll.search('<script charset="UTF-8" src="https://maps.googleapis.com/maps/api/js/ViewportInfoService.GetViewportInfo?1m6&amp;1m2')

In [ ]:
soup.find_all('img', {'alt' : "Map showing where"})

In [ ]:
links_df.iloc[:5]

In [ ]:
links_df['base_link'] = links_df.apply(lambda row: row['links'].split('?')[0], axis=1)

In [ ]:
sum(~links_df['base_link'].duplicated())

In [ ]:
for link in list_of_links[4:6]:
    prop_dict = {}
    driver.get(link)
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, "lxml")
    break

In [ ]:
# soup.find_all('div')
# soup.find_all('div', attrs={'style': "margin-right: 24px;"})

In [ ]:
tag = soup.find_all('div', attrs={'class' : "_czm8crp"})[0]

In [ ]:
tag.get_text()

In [ ]:
# return_url = driver.current_url
# return_url

In [ ]:
# return_url = return_url.replace('adults=1', 'adults=2')
# return_url

In [ ]:
# driver.find_element_by_class_name('_1m76pmy').click()

In [ ]:
driver.find_element_by_xpath('//*[@id="site-content"]/div/div/div[5]/div/div[1]/div').text

In [ ]:
explore = driver.find_element_by_xpath('//*[@id="site-content"]/div/div/div[4]/div/div/div')
explore.find_elements_by_xpath("//div[contains(@itemprop, 'itemListElement')]")[0].find_elements_by_tag_name('meta')[2].get_attribute('content')

In [ ]:
list_of_urls = []

for i in range(4):
    time.sleep(10)
    print(driver.find_element_by_xpath('//*[@id="site-content"]/div/div/div[5]/div/div[1]/div').text)
    explore = driver.find_element_by_xpath('//*[@id="site-content"]/div/div/div[4]/div/div/div')
    props = explore.find_elements_by_xpath("//div[contains(@itemprop, 'itemListElement')]")
    print(explore)
    print(len(props))
    if i == 0:
        # This class only has one instance ont he first page
        driver.find_element_by_class_name('_1m76pmy').click()
    else:
        # If we're not on the first page then choose the second instance of this class
        driver.find_elements_by_class_name('_1m76pmy')[1].click()


In [ ]:
list_of_urls = []

for i in range(4):
#     print(driver.current_url)
    print(driver.find_element_by_xpath('//*[@id="site-content"]/div/div/div[5]/div/div[1]/div').text)
    if i == 0:
        list_of_divs = driver.find_elements_by_xpath("//div[contains(@itemprop, 'itemListElement')]")
    else:
        list_of_divs = driver.find_elements_by_xpath('//*[@id="FMP-target"]/div/div/div/div/div[1]/div/div')
    for prop in list_of_divs:
        list_of_urls.append(prop.find_elements_by_tag_name('meta')[2].get_attribute('content'))
        # print(prop.find_elements_by_tag_name('meta')[2].get_attribute('content'))
        # break
    print('Len of urls:', len(list_of_urls))
    if i == 0:
        # This class only has one instance ont he first page
        driver.find_element_by_class_name('_1m76pmy').click()
    else:
        # If we're not on the first page then choose the second instance of this class
        driver.find_elements_by_class_name('_1m76pmy')[1].click()
    time.sleep(10)


In [ ]:
list_of_divs = driver.find_element_by_xpath("//div[contains(@itemprop, 'itemListElement')]")
list_of_divs

In [ ]:
driver.find_elements_by_xpath('//*[@id="FMP-target"]/div/div/div/div/div[1]/div/div')[0].find_elements_by_tag_name('meta')[2].get_attribute('content')

In [ ]:
for i, v in enumerate(list_of_urls):
    print(i, v)

In [ ]:
print(len(list_of_metas))
for prop in list_of_metas:
    # list_of_metas[0].find_elements_by_tag_name('meta')[2].get_attribute('content')
    print(prop.find_elements_by_tag_name('meta')[2].get_attribute('content'))

In [ ]:
# for meta in list_of_metas:
#     print(meta.value_of_css_property('content'))

In [ ]:
# list_of_metas

In [ ]:
'//*[@id="FMP-target"]/div/div/div/div/div[1]/div/div/meta[3]'

In [ ]:
driver.find_element_by_class_name('_1ip5u88').

In [ ]:
'<a href="/s/seattle/homes?refinement_paths%5B%5D=%2Fhomes&amp;current_tab_id=home_tab&amp;selected_tab_id=home_tab&amp;screen_size=medium&amp;hide_dates_and_guests_filters=true&amp;place_id=ChIJVTPokywQkFQRmtVEaUZlJRA&amp;s_tag=y6o2gwE0&amp;search_type=pagination&amp;section_offset=5&amp;items_offset=36" class="_1ip5u88" aria-busy="false"><div class="_1m76pmy"><svg viewBox="0 0 18 18" role="img" aria-label="Next" focusable="false" style="height: 0.75em; width: 0.75em; display: block; fill: currentcolor;"><path d="m4.29 1.71a1 1 0 1 1 1.42-1.41l8 8a1 1 0 0 1 0 1.41l-8 8a1 1 0 1 1 -1.42-1.41l7.29-7.29z" fill-rule="evenodd"></path></svg></div></a>'

In [ ]:
'//*[@id="FMP-target"]/div/div/div/div/div[1]/div/div/meta[3]'

In [ ]:
'https://www.airbnb.com/s/seattle/homes?refinement_paths%5B%5D=%2Fhomes&current_tab_id=home_tab&selected_tab_id=home_tab&screen_size=large&hide_dates_and_guests_filters=true&place_id=ChIJVTPokywQkFQRmtVEaUZlJRA&s_tag=o6LsNXgV&search_type=pagination'

In [ ]:
while None in [None, None, None]:


In [ ]:
l = [1,1,1,None]
while l.remove(None):
    pass

In [ ]:
l

In [ ]:
def get_superhost(spans):
    superhost = False
    sh = re.compile(r'^\w+ is a Superhost')
    for span in spans:
        if sh.match(span.text):
            superhost = True
    return superhost
get_superhost(soup.find_all('span', {'class' : "_1p3joamp"}))